# CNC Mill Tool Wear
- https://www.kaggle.com/shasun/tool-wear-detection-in-cnc-mill
- Predict "Machining_Process"

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import os
path_parent = os.path.dirname(os.getcwd())
saveFolder = os.path.join(path_parent, 'Topic 02 - Classification', 'Data_CNC_Processed')
print(saveFolder)
#saveFolder = '..\Topic 02 - Classification\Data_CNC_Processed'
filename = 'pred_machining_process'
compression = 'gzip'
df = pd.read_pickle(f'{saveFolder}/{filename}.pkl',compression=compression)
df.head(3)

f:\Work\Coding\Class\ai-class-2564-1\Topic 02 - Classification\Data_CNC_Processed


,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,...,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process,feedrate,clamp_pressure
0,198.0,0.0,0.00,198.0,0.0,0.000000,0.18,0.0207,329.0,2.77,...,329.0,0.0,6.960000e-07,12.0,1.0,0.0,50.0,7,6,4.0
1,198.0,-10.8,-350.00,198.0,-13.6,-358.000000,-10.90,0.1860,328.0,23.30,...,328.0,0.0,-5.270000e-07,12.0,1.0,4.0,50.0,7,6,4.0
2,196.0,-17.8,-6.25,196.0,-17.9,-0.000095,-8.59,0.1400,328.0,30.60,...,328.0,0.0,9.100000e-07,12.0,1.0,7.0,50.0,7,6,4.0


In [3]:
colsX = np.array([item for item in df.columns if item not in 'Machining_Process'])
X = df[colsX].values
y = df['Machining_Process'].values

In [4]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)


## Contruct pipeline

In [5]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear', penalty='l1', C=0.1, multi_class='ovr')

var = VarianceThreshold(threshold=0)
l1 = SelectFromModel(estimator=lr, norm_order=1, max_features=20)
clf = RF(n_estimators=50)

# Construct pipeline object
pipe = Pipeline([
    ('var', var)
    ('scl', StandardScaler()),
    ('l1',l1),
    ('clf', clf)])

<>:15: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:15: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<ipython-input-5-f492bab84e55>:15: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('var', var)


TypeError: 'tuple' object is not callable

## Comparing training and validation accuracy


In [ ]:
# With cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=pipe,
                         X=X_train,
                         y=y_train,
                         cv=5,
                         n_jobs=1)

arrayPrint =   ', '.join([f'{el:3.2f}' for el in scores])
print(f"Score:", arrayPrint)
print("\nCrossed-validation accuracy")  
print(f"Mean:{np.mean(scores):6.3f}")
print(f"Std:{np.std(scores):6.3f}")

Score: 0.64, 0.62, 0.64, 0.71, 0.67

Crossed-validation accuracy
Mean: 0.656
Std: 0.030
